In [ ]:
import tensorflow as tf
import numpy as np
import math

In [ ]:
# Helper function
def batches(batch_size, features, labels):
    assert len(features) == len(labels)
    
    output_batches = []
    for start_i in range(0, len(features), batch_size):
        end_i = start_i + batch_size
        batch = [features[start_i: end_i], labels[start_i: end_i]]
        output_batches.append(batch)
        
    return output_batches

def encode_onehot(labels, n_class):
    onehot_labels = np.zeros((len(labels), n_class))
    for label, onehot_label in zip(labels, onehot_labels):
        onehot_label[label-1] = 1
    return onehot_labels

In [ ]:
n_input = 784  # MNIST data input (28x28) <= num of nodes in input layer
n_classes = 10 # MNIST total classes 0-9 digits <= num of nodes in output layer

In [ ]:
# Load MNIST data sets
mnist = tf.keras.datasets.mnist

(train_features, train_labels), (test_features, test_labels) = mnist.load_data()

train_features = train_features/255.0
test_features = test_features/255.0

train_features = np.reshape(train_features, (len(train_features), n_input))
test_features = np.reshape(test_features, (len(test_features), n_input))

train_labels = encode_onehot(train_labels, n_classes)
test_labels = encode_onehot(test_labels, n_classes)

In [ ]:
# construct the single layer neural network
features = tf.placeholder(tf.float32, [None, n_input])
labels = tf.placeholder(tf.float32, [None, n_classes])

w1 = tf.Variable(tf.random_normal([n_input, n_classes]))
b1 = tf.Variable(tf.random_normal([n_classes]))

logits = tf.add(tf.matmul(features, w1), b1)

learning_rate = tf.placeholder(tf.float32)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# calculate accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
batch_size = 128
epochs = 80
lr = 0.001

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    
    for i in range(epochs):
        for bat_features, bat_labels in batches(batch_size, train_features, train_labels):
            train_feed_dict = {
                features: bat_features,
                labels: bat_labels, 
                learning_rate: lr}
            sess.run(optimizer, feed_dict=train_feed_dict)
        
    test_accuracy = sess.run(
        accuracy,
        feed_dict={features: test_features, labels: test_labels})
    
print('Test Accuracy: {}'.format(test_accuracy))

In [ ]:
#from tensorflow.python.client import device_lib
#device_lib.list_local_devices()